In [1]:
!pip install tensorflow numpy


In [2]:
# Create a directory for the project
import os
os.makedirs('adaptive_theme_project', exist_ok=True)

# Write HTML content
with open('adaptive_theme_project/index.html', 'w') as f:
    f.write("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Adaptive Theme Changer</title>
        <link id="theme-style" rel="stylesheet" href="light.css">
    </head>
    <body>
        <h1>Welcome to Adaptive Theme Changer!</h1>
        <button onclick="switchTheme()">Switch Theme</button>
        <script src="theme.js"></script>
    </body>
    </html>
    """)

# Write CSS content for light mode
with open('adaptive_theme_project/light.css', 'w') as f:
    f.write("""
    body {
        background-color: white;
        color: black;
    }
    """)

# Write CSS content for dark mode
with open('adaptive_theme_project/dark.css', 'w') as f:
    f.write("""
    body {
        background-color: black;
        color: white;
    }
    """)

# Write JavaScript for theme switching
with open('adaptive_theme_project/theme.js', 'w') as f:
    f.write("""
    function switchTheme() {
        const currentTheme = document.getElementById('theme-style').getAttribute('href');
        const newTheme = currentTheme === 'light.css' ? 'dark.css' : 'light.css';
        document.getElementById('theme-style').setAttribute('href', newTheme);
    }
    """)


In [3]:
import numpy as np

# Simulate user behavior data
# Columns: [Time of Day (0-23), Theme Preference (0: Light, 1: Dark)]
data = np.array([
    [8, 0], [9, 0], [18, 1], [19, 1],  # Morning -> Light, Evening -> Dark
    [21, 1], [22, 1], [7, 0], [6, 0]   # Early Morning -> Light
])

# Save data for training
np.save('adaptive_theme_project/user_data.npy', data)


In [5]:
import tensorflow as tf

# Load the data
data = np.load('adaptive_theme_project/user_data.npy')
X, y = data[:, 0], data[:, 1]

# Normalize inputs
X = X / 23.0

# Create a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, verbose=0)

# Save the model
model.save('adaptive_theme_project/theme_model.keras') # Add .keras extension to the filename


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
!pip install tensorflowjs
!tensorflowjs_converter --input_format=tf_saved_model adaptive_theme_project/theme_model adaptive_theme_project/theme_model_js


INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 81.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully unin

2024-11-23 23:01:15.807139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732402875.833278    1838 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732402875.841000    1838 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
  File "/usr/local/lib/python3.10/dist-packages/tensorflowjs/converters/converter.py", line 959, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib/python3.10/dist-package

In [8]:
from flask import Flask, render_template_string
import datetime

app = Flask(__name__)

@app.route('/')
def home():
    # Get current hour
    current_hour = datetime.datetime.now().hour

    # Decide which theme to serve (light in the morning, dark in the evening)
    if 6 <= current_hour < 18:
        theme = 'light.css'  # Daytime: light theme
    else:
        theme = 'dark.css'   # Nighttime: dark theme

    # Read the HTML file as a string and inject the selected theme
    with open('adaptive_theme_project/index.html', 'r') as file:
        html_content = file.read()

    return render_template_string(html_content, theme=theme)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [9]:
# Create the Flask application file `app.py`
with open('adaptive_theme_project/app.py', 'w') as f:
    f.write("""
from flask import Flask, render_template_string
import datetime

app = Flask(__name__)

@app.route('/')
def home():
    # Get current hour
    current_hour = datetime.datetime.now().hour

    # Decide which theme to serve (light in the morning, dark in the evening)
    if 6 <= current_hour < 18:
        theme = 'light.css'  # Daytime: light theme
    else:
        theme = 'dark.css'   # Nighttime: dark theme

    # Read the HTML file as a string and inject the selected theme
    with open('adaptive_theme_project/index.html', 'r') as file:
        html_content = file.read()

    return render_template_string(html_content, theme=theme)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
    """)


In [10]:
!pip install Flask

In [11]:
!FLASK_APP=adaptive_theme_project/app.py flask run --host=0.0.0.0 --port=8000 --no-reload

 * Serving Flask app 'adaptive_theme_project/app.py'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
Press CTRL+C to quit
^C


In [12]:
!pip install flask-ngrok

In [13]:
from flask import Flask, render_template_string
import datetime
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # This will expose your app to the internet

@app.route('/')
def home():
    # Get current hour
    current_hour = datetime.datetime.now().hour

    # Decide which theme to serve (light in the morning, dark in the evening)
    if 6 <= current_hour < 18:
        theme = 'light.css'  # Daytime: light theme
    else:
        theme = 'dark.css'   # Nighttime: dark theme

    # Read the HTML file as a string and inject the selected theme
    with open('adaptive_theme_project/index.html', 'r') as file:
        html_content = file.read()

    return render_template_string(html_content, theme=theme)

if __name__ == '__main__':
    # Just call app.run() without additional arguments
    app.run()




 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [14]:
!python adaptive_theme_project/app.py

 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C
